In [ ]:
!pip install rasterio
!pip install osmnx
!pip install mapclassify folium matplotlib

In [ ]:
import osmnx as ox
from osgeo import gdal
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from shapely.geometry import LineString
from rasterio.mask import mask
import rasterio

### Wilmington, North Carolina, USA

In [ ]:
place_name = "Wilmington, North Carolina, USA"
G = ox.graph_from_place(place_name, network_type='drive')
fig,ax = ox.plot_graph(G)
fig.savefig("Wilmington, North Carolina, USA.png")

### Hyde County, North Carolina, USA

In [ ]:
place_name = "Hyde County, NC, USA" # Changed the place name to be more specific and corrected potential typo
graph = ox.graph_from_place(place_name, network_type='drive')
fig,ax = ox.plot_graph(graph)
fig.savefig("Hyde County, North Carolina, USA.png")

In [ ]:
Wilmington_drive = ox.speed.add_edge_speeds(G)
Wilmington_drive = ox.speed.add_edge_travel_times(G)

In [ ]:
Wilmington_drive = ox.elevation.add_node_elevations_raster(Wilmington_drive, 'Wilmington, NC_Copernicus_DSM_COG_10_N34_00_W078_00_HAND.tif', cpus=1)
Wilmington_drive = ox.add_edge_grades(Wilmington_drive, add_absolute=True)

In [ ]:
Wilmington_drive_node, Wilmington_drive_edge = ox.graph_to_gdfs(Wilmington_drive)
Wilmington_drive_node

###  Identifying the Highest and Lowest Elevation Points

In [ ]:
# Assuming Wilmington_drive_node is your DataFrame
highest_elevation = Wilmington_drive_node['elevation'].max()
lowest_elevation = Wilmington_drive_node['elevation'].min()

# Output the highest and lowest elevation values
print("Highest elevation value:", highest_elevation)
print("Lowest elevation value:", lowest_elevation)


In [ ]:
Wilmington_drive_node = Wilmington_drive_node.reset_index()
print(Wilmington_drive_node.columns)
Wilmington_drive_node

In [ ]:
# Reset the index to turn 'u', 'v', 'key' into columns
Wilmington_drive_edge = Wilmington_drive_edge.reset_index()

# Now 'u', 'v', and 'key' will be columns in the DataFrame
print(Wilmington_drive_edge.columns)

In [ ]:
Wilmington_drive_edge


In [ ]:
pip install geopandas

### Nodes Colored by Elevation

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Create a GeoDataFrame
df = pd.DataFrame(Wilmington_drive_node)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x'], df['y']))

# Plotting the GeoDataFrame
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
gdf.plot(column='elevation', ax=ax, legend=True, cmap='viridis', markersize=100, legend_kwds={'label': "Elevation (m)"})

# Customize plot appearance
plt.title('Nodes Colored by Elevation')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.savefig("Wilmington, NC Nodes Colored by Elevation.png")
plt.show()

In [ ]:
print(Wilmington_drive_edge.columns)


In [ ]:
 import pandas as pd

# Load the data
# Wilmington_drive_edge = pd.read_csv('Wilmington_drive_edge.csv')  # already loaded
# Wilmington_drive_node = pd.read_csv('Wilmington_drive_node.csv')  # already loaded

# Check column names in both DataFrames
print("Wilmington_drive_node columns:", Wilmington_drive_node.columns)
print("Wilmington_drive_edge columns:", Wilmington_drive_edge.columns)

# Assuming the node ID column in san_drive_node is 'node_id' or 'osmid'
node_column = 'osmid'  # Replace this with the actual name of the node ID column from your output

# Merge the node data to get the elevation of both u and v endpoints
Wilmington_drive_edge = Wilmington_drive_edge.merge(
    Wilmington_drive_node[[node_column, 'elevation']],
    left_on='u',
    right_on=node_column,
    how='left'
).rename(columns={'elevation': 'elevation_u'})

Wilmington_drive_edge = Wilmington_drive_edge.merge(
    Wilmington_drive_node[[node_column, 'elevation']],
    left_on='v',
    right_on=node_column,
    how='left'
).rename(columns={'elevation': 'elevation_v'})

# Now check the merged DataFrame
print(Wilmington_drive_edge[['u', 'v', 'elevation_u', 'elevation_v']].head())


### Flooded_threshold = 1

In [ ]:
# Define thresholds and color mapping
flooded_threshold = 1  # Elevation <= 1 meters are considered flooded

prone_threshold = 5   # Elevation between 1 and 9 meters are prone to flooding
safe_threshold = 8   # Elevation >= 8 meters are safe 
node_color_map = {
    'flooded': 'red',
    'prone': 'blue',
    'safe': 'green',
    'unknown': 'gray'
}

# Function to classify nodes based on elevation
def classify_node_by_elevation(elevation):
    if np.isnan(elevation):
        return 'unknown'  # Handle NaN values
    elif elevation <= flooded_threshold:
        return 'flooded'
   # elif flooded_threshold < elevation <= less_flooded_threshold:
        #return 'less_flooded'  # New classification for less flooded areas
    elif flooded_threshold < elevation <= prone_threshold:
        return 'prone'
    else:
        return 'safe'

# Now you can apply this function to classify the nodes
Wilmington_drive_node['flood_status'] = Wilmington_drive_node['elevation'].apply(classify_node_by_elevation)
Wilmington_drive_node['color'] = Wilmington_drive_node['flood_status'].map(node_color_map)


# Load the CSV file into a DataFrame (assuming it contains columns for 'elevation' and 'osmid')
csv_file = 'Wilmington_drive_edge.csv'  # Replace with the actual CSV file path
#san_drive_node = pd.read_csv(csv.file)

# Apply classification
Wilmington_drive_node['flood_status'] = Wilmington_drive_node['elevation'].apply(classify_node_by_elevation)
Wilmington_drive_node['color'] = Wilmington_drive_node['flood_status'].map(node_color_map)

# Extract the flooded data
flooded_data = Wilmington_drive_node[Wilmington_drive_node['flood_status'] == 'flooded']

# Save the flooded data to a new CSV file (optional)
flooded_data.to_csv('Flooded_areas.csv', index=False)

# Plotting function
def plot_flooded_areas():
    fig, ax = plt.subplots(figsize=(12, 12))

    # Plot the road network (background)
    Wilmington_drive_edge.plot(ax=ax, color='black', linewidth=0.5, alpha=0.5, label='Road Network')

    # Plot the nodes color-coded by flood status
    for status, color in node_color_map.items():
        subset = Wilmington_drive_node[Wilmington_drive_node['flood_status'] == status]
        if not subset.empty:
            subset.plot(ax=ax, color=color, markersize=20, alpha=0.8, label=status)

    # Set aspect ratio to auto to avoid errors
    ax.set_aspect('auto')

    # Add title and labels
    plt.title("Flood Risk Based on Elevation")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

    # Create custom legend handles
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=status)
               for status, color in node_color_map.items()]

    # Add the legend to the plot
    ax.legend(handles=handles, title="Flood Status", loc='upper right')
# Call the function to plot
plot_flooded_areas()
plt.savefig("Flood Risk Based on Elevation_flooded_threshold = 1.png")

# Show the plot
plt.show()

### Flood_risk_status

In [ ]:
def label_flood_risk(df, threshold):
    """
    Labels each edge in the dataframe based on the elevation of u and v endpoints.

    Parameters:
    df : pandas.DataFrame
        DataFrame containing 'elevation_u' and 'elevation_v' columns.
    threshold : float
        The elevation threshold below which nodes are considered flooded.

    Returns:
    pandas.DataFrame
        DataFrame with a new column 'flood_risk' indicating flood status.
    """
    def classify_edge(elevation_u, elevation_v):
        if elevation_u < threshold and elevation_v < threshold:
            return 'flooded'
        elif elevation_u >= threshold and elevation_v >= threshold:
            return 'safe'
        else:
            return 'partially-flooded'

    df['flood_risk'] = df.apply(lambda row: classify_edge(row['elevation_u'], row['elevation_v']), axis=1)
    return df

# Example usage:
Wilmington_drive_edge = label_flood_risk(Wilmington_drive_edge, threshold=10)
print(Wilmington_drive_edge[['u', 'v','elevation_u', 'elevation_v', 'flood_risk']].head())


### Interactive Flood Risk Map for Wilmington: Visualizing Road Networks and Node Status

In [ ]:
import folium
from shapely.geometry import Point

# Initialize the folium map
m = folium.Map(location=[34.225, -77.944], zoom_start=12)

# Add the road network to the map
for _, row in Wilmington_drive_edge.iterrows():
    # Ensure the geometry column contains valid LineString objects
    if row['geometry'].geom_type == 'LineString':
        folium.PolyLine(
            locations=[(point[1], point[0]) for point in row['geometry'].coords],  # Reverse for (latitude, longitude)
            color="black",
            weight=1,
            opacity=0.5,
        ).add_to(m)

# Add nodes color-coded by flood status
for _, row in Wilmington_drive_node.iterrows():
    # Ensure the geometry column contains valid Point objects
    if isinstance(row['geometry'], Point):
        folium.CircleMarker(
            location=(row['geometry'].y, row['geometry'].x),  # Latitude, Longitude
            radius=5,
            color=node_color_map[row['flood_status']],
            fill=True,
            fill_color=node_color_map[row['flood_status']],
            fill_opacity=0.5,
            tooltip=(
                f"Flood Status: {row['flood_status']}<br>"
                f"Elevation: {row['elevation']}m"
            ),
        ).add_to(m)

# Save the map to an HTML file
m.save("Wilmington_flood_risk_map_threshold1.html")

# Display the map in a Jupyter Notebook (optional)
m


### Flooded_threshold = 3

In [ ]:
# Define thresholds and color mapping
flooded_threshold = 3  # Elevation <= 3 meters are considered flooded

prone_threshold = 12    # Elevation between 2 and 12 meters are prone to flooding
safe_threshold = 15   # Elevation >= 15 meters are safe
node_color_map = {
    'flooded': 'red',
    'prone': 'blue',
    'safe': 'green',
    'unknown': 'gray'
}

# Function to classify nodes based on elevation
def classify_node_by_elevation(elevation):
    if np.isnan(elevation):
        return 'unknown'  # Handle NaN values
    elif elevation <= flooded_threshold:
        return 'flooded'
   # elif flooded_threshold < elevation <= less_flooded_threshold:
        #return 'less_flooded'  # New classification for less flooded areas
    elif flooded_threshold < elevation <= prone_threshold:
        return 'prone'
    else:
        return 'safe'

# Now you can apply this function to classify the nodes
Wilmington_drive_node['flood_status'] = Wilmington_drive_node['elevation'].apply(classify_node_by_elevation)
Wilmington_drive_node['color'] = Wilmington_drive_node['flood_status'].map(node_color_map)


# Load the CSV file into a DataFrame (assuming it contains columns for 'elevation' and 'osmid')
csv_file = 'Wilmington_drive_edge.csv'  # Replace with the actual CSV file path
#san_drive_node = pd.read_csv(csv.file)

# Apply classification
Wilmington_drive_node['flood_status'] = Wilmington_drive_node['elevation'].apply(classify_node_by_elevation)
Wilmington_drive_node['color'] = Wilmington_drive_node['flood_status'].map(node_color_map)

# Extract the flooded data
flooded_data = Wilmington_drive_node[Wilmington_drive_node['flood_status'] == 'flooded']

# Plotting function
def plot_flooded_areas():
    fig, ax = plt.subplots(figsize=(12, 12))

    # Plot the road network (background)
    Wilmington_drive_edge.plot(ax=ax, color='black', linewidth=0.5, alpha=0.5, label='Road Network')

    # Plot the nodes color-coded by flood status
    for status, color in node_color_map.items():
        subset = Wilmington_drive_node[Wilmington_drive_node['flood_status'] == status]
        if not subset.empty:
            subset.plot(ax=ax, color=color, markersize=20, alpha=0.8, label=status)

    # Set aspect ratio to auto to avoid errors
    ax.set_aspect('auto')

    # Add title and labels
    plt.title("Flood Risk Based on Elevation")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

    # Create custom legend handles
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=status)
               for status, color in node_color_map.items()]

    # Add the legend to the plot
    ax.legend(handles=handles, title="Flood Status", loc='upper right')    
# Call the function to plot
plot_flooded_areas()
plt.savefig("Flood Risk Based on Elevation_flooded_threshold = 2.png")
# Show the plot
plt.show()

In [ ]:
import folium

# Initialize the folium map
m = folium.Map(location=[34.225, -77.944], zoom_start=12)

# Add the road network to the map
for _, row in Wilmington_drive_edge.iterrows():
    folium.PolyLine(
        locations=[(point[1], point[0]) for point in row['geometry'].coords],  # Reverse order for latitude, longitude
        color="black",
        weight=1,
        opacity=0.5,
    ).add_to(m)

# Add nodes with color-coded flood statuses
for _, row in Wilmington_drive_node.iterrows():
    folium.CircleMarker(
        location=(row.geometry.y, row.geometry.x),
        radius=5,
        color=node_color_map[row['flood_status']],
        fill=True,
        fill_color=node_color_map[row['flood_status']],
        fill_opacity=0.5,
        tooltip=f"Flood Status: {row['flood_status']}\nElevation: {row['elevation']}m",
    ).add_to(m)

# Save the map or display it
m.save("Wilmington_flood_risk_map_threshold3.html")
m


### Flooded_threshold = 7

In [ ]:
# Define thresholds and color mapping
flooded_threshold = 7  # Elevation <= 7 meters are considered flooded

prone_threshold = 14    # Elevation between 7 and 14 meters are prone to flooding
safe_threshold = 16   # Elevation >= 16 meters are safe
node_color_map = {
    'flooded': 'red',
    'prone': 'blue',
    'safe': 'green',
    'unknown': 'gray'
}

# Function to classify nodes based on elevation
def classify_node_by_elevation(elevation):
    if np.isnan(elevation):
        return 'unknown'  # Handle NaN values
    elif elevation <= flooded_threshold:
        return 'flooded'
   # elif flooded_threshold < elevation <= less_flooded_threshold:
        #return 'less_flooded'  # New classification for less flooded areas
    elif flooded_threshold < elevation <= prone_threshold:
        return 'prone'
    else:
        return 'safe'

# Now you can apply this function to classify the nodes
Wilmington_drive_node['flood_status'] = Wilmington_drive_node['elevation'].apply(classify_node_by_elevation)
Wilmington_drive_node['color'] = Wilmington_drive_node['flood_status'].map(node_color_map)


# Load the CSV file into a DataFrame (assuming it contains columns for 'elevation' and 'osmid')
csv_file = 'Wilmington_drive_edge.csv'  # Replace with the actual CSV file path
#san_drive_node = pd.read_csv(csv.file)

# Apply classification
Wilmington_drive_node['flood_status'] = Wilmington_drive_node['elevation'].apply(classify_node_by_elevation)
Wilmington_drive_node['color'] = Wilmington_drive_node['flood_status'].map(node_color_map)

# Extract the flooded data
flooded_data = Wilmington_drive_node[Wilmington_drive_node['flood_status'] == 'flooded']


# Plotting function
def plot_flooded_areas():
    fig, ax = plt.subplots(figsize=(12, 12))

    # Plot the road network (background)
    Wilmington_drive_edge.plot(ax=ax, color='black', linewidth=0.5, alpha=0.5, label='Road Network')

    # Plot the nodes color-coded by flood status
    for status, color in node_color_map.items():
        subset = Wilmington_drive_node[Wilmington_drive_node['flood_status'] == status]
        if not subset.empty:
            subset.plot(ax=ax, color=color, markersize=20, alpha=0.8, label=status)

    # Set aspect ratio to auto to avoid errors
    ax.set_aspect('auto')

    # Add title and labels
    plt.title("Flood Risk Based on Elevation")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

    # Create custom legend handles
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=status)
               for status, color in node_color_map.items()]

    # Add the legend to the plot
    ax.legend(handles=handles, title="Flood Status", loc='upper right')
# Call the function to plot
plot_flooded_areas()
plt.savefig("Flood Risk Based on Elevation_flooded_threshold = 7.png")
# Show the plot
plt.show()

### check coordinates and flood status

In [ ]:
import folium

# Initialize the folium map
m = folium.Map(location=[34.225, -77.944], zoom_start=12)

# Add the road network to the map
for _, row in Wilmington_drive_edge.iterrows():
    folium.PolyLine(
        locations=[(point[1], point[0]) for point in row['geometry'].coords],
        color="black",
        weight=1,
        opacity=0.5,
    ).add_to(m)

# Add nodes with color-coded flood statuses
for _, row in Wilmington_drive_node.iterrows():
    # Debug print to check coordinates and flood status
    print(f"Latitude: {row.geometry.y}, Longitude: {row.geometry.x}")
    print(f"Flood Status: {row['flood_status']}")


In [ ]:
import folium

# Initialize the folium map centered on the given coordinates with a zoom level
m = folium.Map(location=[34.225, -77.944], zoom_start=12, tiles='Stamen Terrain')

# Add Google Maps as a tile layer
folium.TileLayer(
    tiles='https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Maps',
    overlay=False,
    control=True
).add_to(m)

# Add the road network to the map
for _, row in Wilmington_drive_edge.iterrows():
    folium.PolyLine(
        locations=[(point[1], point[0]) for point in row['geometry'].coords],
        color="black",
        weight=1,
        opacity=0.5,
    ).add_to(m)

# Add nodes with color-coded flood statuses
for _, row in Wilmington_drive_node.iterrows():
    # Set the color based on flood status
    if row['flood_status'] == 'flooded':
        marker_color = 'red'
    elif row['flood_status'] == 'prone':
        marker_color = 'blue'
    else:
        marker_color = 'green'
    
    # Add marker with popup showing the flood status and additional dynamic info
    folium.Marker(
        location=(row.geometry.y, row.geometry.x),
        icon=folium.Icon(color=marker_color)
    ).add_to(m).add_child(
        folium.Popup(f"Road Status: {row['flood_status']}\nTraffic Rerouting Info: [Dynamic Information]")
    )

# Add layer control for Google Maps
folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save("flood_risk_map_with_google.html")
m

In [ ]:
Wilmington_drive_edge['grade_abs'].unique()


### Wilmington Road Network with Flood Risk Classification by Grade

In [ ]:
# Define the criteria based on grade absolute
flooded_threshold = 0.009   # Roads with grade absolute below 2% are very flat (flooded)
prone_threshold = 0.052     # Roads with grade absolute between 2% and 5% are prone to flood
safe_threshold = 0.222   # Roads with grade absolute above 5% are generally safe

# Classify the roads based on grade absolute
def classify_road_by_grade(grade_abs):
    if grade_abs < flooded_threshold:
        return 'flooded'
    elif flooded_threshold <= grade_abs < prone_threshold:
        return 'prone'
    else:
        return 'safe'

# Apply classification
Wilmington_drive_edge['flood_status'] = Wilmington_drive_edge['grade_abs'].apply(classify_road_by_grade)

# Assign colors based on classification
color_map = {
    'flooded': 'red',
    'prone': 'blue',
    'safe': 'green'
}
Wilmington_drive_edge['color'] = Wilmington_drive_edge['flood_status'].map(color_map)

# Plot the road network with colors based on the updated flood classification
fig, ax = plt.subplots(figsize=(12, 12))

for status, color in color_map.items():
    subset = Wilmington_drive_edge[Wilmington_drive_edge['flood_status'] == status]
    if not subset.empty:
        subset.plot(ax=ax, color=color, label=status, linewidth=0.5, alpha=0.7)

# Add legend and title
plt.legend(title="Flood Status by Grade")
plt.title("Wilmington Road Network with Flood Risk Classification by Grade")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.savefig("Wilmington Road Network with Flood Risk Classification by Grade.png")
plt.show()

### Hyde County

In [ ]:
place_name = "Hyde County, North Carolina, USA"
G = ox.graph_from_place(place_name, network_type='drive')
ox.plot_graph(G)
plt.savefig("Hyde County, North Carolina, USA.png")
# Show the plot
plt.show()

In [ ]:
import osmnx as ox

# Download the road network for Hyde County
place_name = "Hyde County, North Carolina, USA"
Hyde_County_drive = ox.graph_from_place(place_name, network_type='drive')


In [ ]:
import osmnx as ox
from osgeo import gdal
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from shapely.geometry import LineString
from rasterio.mask import mask
import rasterio

In [ ]:
Hyde_County_drive = ox.speed.add_edge_speeds(G)
Hyde_County_drive = ox.speed.add_edge_travel_times(Hyde_County_drive)

In [ ]:
Hyde_County_drive = ox.elevation.add_node_elevations_raster(Hyde_County_drive, 'Hyde county Copernicus_DSM_COG_10_N35_00_W077_00_HAND.tif', cpus=1)
Hyde_County_drive = ox.add_edge_grades(Hyde_County_drive, add_absolute=True)

In [ ]:
Hyde_County_drive_node, Hyde_County_drive_edge = ox.graph_to_gdfs(Hyde_County_drive)
Hyde_County_drive_node
Wilmington_drive_node, Wilmington_drive_edge = ox.graph_to_gdfs(Wilmington_drive)
Wilmington_drive_node

###  Identifying the Highest and Lowest Elevation Points

In [ ]:
# Assuming Hyde_County_drive_node is your DataFrame
highest_elevation = Hyde_County_drive_node['elevation'].max()
lowest_elevation = Hyde_County_drive_node['elevation'].min()

# Output the highest and lowest elevation values
print("Highest elevation value:", highest_elevation)
print("Lowest elevation value:", lowest_elevation)


In [ ]:
# Reset the index to turn 'u', 'v', 'key' into columns
Hyde_County_drive_edge = Hyde_County_drive_edge.reset_index()

# Now 'u', 'v', and 'key' will be columns in the DataFrame
print(Hyde_County_drive_edge.columns)

In [ ]:
Hyde_County_drive_edge

### Hyde County Nodes Colored by Elevation

In [ ]:
# Create a GeoDataFrame
df = pd.DataFrame(Hyde_County_drive_node)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x'], df['y']))

# Plotting the GeoDataFrame
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
gdf.plot(column='elevation', ax=ax, legend=True, cmap='viridis', markersize=100, legend_kwds={'label': "Elevation (m)"})

# Customize plot appearance
plt.title('Nodes Colored by Elevation')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
print(Hyde_County_drive_edge.columns)

### Hyde County Flood Risk Based on Elevation

In [ ]:
# Define thresholds and color mapping
flooded_threshold = 1  # Elevation <= 1 meters are considered flooded
prone_threshold = 12    # Elevation between 1 and 12 meters are prone to flooding
safe_threshold = 13   # Elevation >= 13 meters are safe
node_color_map = {
    'flooded': 'red',
    'prone': 'blue',
    'safe': 'green',
    'unknown': 'gray'
}

# Function to classify nodes based on elevation
def classify_node_by_elevation(elevation):
    if np.isnan(elevation):
        return 'unknown'  # Handle NaN values
    elif elevation <= flooded_threshold:
        return 'flooded'
   # elif flooded_threshold < elevation <= less_flooded_threshold:
        #return 'less_flooded'  # New classification for less flooded areas
    elif flooded_threshold < elevation <= prone_threshold:
        return 'prone'
    else:
        return 'safe'

# Now you can apply this function to classify the nodes
Hyde_County_drive_node['flood_status'] = Hyde_County_drive_node['elevation'].apply(classify_node_by_elevation)
Hyde_County_drive_node['color'] = Hyde_County_drive_node['flood_status'].map(node_color_map)

# Apply classification
Hyde_County_drive_node['flood_status'] = Hyde_County_drive_node['elevation'].apply(classify_node_by_elevation)
Hyde_County_drive_node['color'] = Hyde_County_drive_node['flood_status'].map(node_color_map)

# Extract the flooded data
flooded_data = Hyde_County_drive_node[Hyde_County_drive_node['flood_status'] == 'flooded']

# Plotting function
def plot_flooded_areas():
    fig, ax = plt.subplots(figsize=(12, 12))

    # Plot the road network (background)
    Hyde_County_drive_edge.plot(ax=ax, color='black', linewidth=0.5, alpha=0.5, label='Road Network')

    # Plot the nodes color-coded by flood status
    for status, color in node_color_map.items():
        subset = Hyde_County_drive_node[Hyde_County_drive_node['flood_status'] == status]
        if not subset.empty:
            subset.plot(ax=ax, color=color, markersize=20, alpha=0.8, label=status)

    # Set aspect ratio to auto to avoid errors
    ax.set_aspect('auto')

    # Add title and labels
    plt.title("Hyde County Flood Risk Based on Elevation")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

    # Create custom legend handles
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=status)
               for status, color in node_color_map.items()]

    # Add the legend to the plot
    ax.legend(handles=handles, title="Flood Status", loc='upper right')
# Call the function to plot
plot_flooded_areas()
plt.savefig("Hyde County Flood Risk Based on Elevation.png")
# Show the plot
plt.show()

### Hyde County Flood Risk and Road Network Visualization

In [ ]:
import folium

# Initialize the folium map for Hyde County
m = folium.Map(location=[35.559, -76.297], zoom_start=12)  # Coordinates updated for Hyde County

# Add the road network to the map
for _, row in Hyde_County_drive_edge.iterrows():
    folium.PolyLine(
        locations=[(point[1], point[0]) for point in row['geometry'].coords],
        color="black",
        weight=1,
        opacity=0.5,
    ).add_to(m)

# Add nodes with color-coded flood statuses as part of the digital twin model
for _, row in Hyde_County_drive_node.iterrows():
    # Create a color map for flood status dynamically (ensure `node_color_map` is defined)
    node_color = node_color_map.get(row['flood_status'], 'gray')  # Default to gray if status is missing

    folium.CircleMarker(
        location=(row.geometry.y, row.geometry.x),
        radius=5,
        color=node_color,
        fill=True,
        fill_color=node_color,
        fill_opacity=0.6,
        tooltip=(
            f"<strong>Flood Status:</strong> {row['flood_status']}<br>"
            f"<strong>Elevation:</strong> {row['elevation']} m"
        ),
    ).add_to(m)



# Save the map or display it
m.save("hyde_county_flood_risk_map.html")
m


In [ ]:
Hyde_County_drive_edge['grade_abs'].unique()

In [ ]:
# Adjusted criteria based on grade absolute
flooded_threshold = 0.0034   # Roads with grade absolute below 1% are very flat (flooded)
prone_threshold = 0.04     # Roads with grade absolute between 2% and 5% are prone to flood
safe_threshold = 0.05      # Roads with grade absolute above 5% are generally safe

# Classify the roads based on grade absolute
def classify_road_by_grade(grade_abs):
    if grade_abs < flooded_threshold:
        return 'flooded'
    elif flooded_threshold <= grade_abs < prone_threshold:
        return 'prone'
    else:
        return 'safe'

# Apply classification to Hyde County drive edges
Hyde_County_drive_edge['flood_status'] = Hyde_County_drive_edge['grade_abs'].apply(classify_road_by_grade)

# Assign colors based on classification
color_map = {
    'flooded': 'red',
    'prone': 'blue',
    'safe': 'green'
}
Hyde_County_drive_edge['color'] = Hyde_County_drive_edge['flood_status'].map(color_map)

# Plot the road network with colors based on the updated flood classification
fig, ax = plt.subplots(figsize=(12, 12))

for status, color in color_map.items():
    subset = Hyde_County_drive_edge[Hyde_County_drive_edge['flood_status'] == status]
    if not subset.empty:
        subset.plot(ax=ax, color=color, label=status, linewidth=0.5, alpha=0.7)

# Add legend and title
plt.legend(title="Flood Status by Grade")
plt.title("Hyde County Road Network with Flood Risk Classification by Grade")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.savefig("Hyde County Road Network with Flood Risk Classification by Grade.png")
plt.show()
